# Advanced Python - Building Scalable Applications

### Module 7

#### Network socket programming
 -  Using ```asyncio``` and ```trio``` for concurrent socket programming

#### Web (RESTful) API using Python
 - An overview on RESTful API architecture
 - Consuming web API using python requests and httpx
 - An overview on WSGI and ASGI
 - WSGI / ASGI servers: ```gunicorn``` and ```uvicorn```
 - An overview on ```starlette``` ASGI framework
 - Creating and exposing web API using ```FastAPI``` framework
 - Discovering web APIs using ```/docs``` (Open-API) and ```/redocs``` interface
 - Creating validating models using ```pydantic```
 - Security Considerations: Authentication and Authorization techniques


In [4]:
class User:
    def greet(self):
        print("greetings...")

u = User()
u.greet()

hasattr(u, "greet")
g = getattr(u, "greet")
g()

greetings...
greetings...


In [5]:
# Handling binary data over text I/O streams

outs = open("/tmp/test.dat", "w")
outs

<_io.TextIOWrapper name='/tmp/test.dat' mode='w' encoding='UTF-8'>

In [15]:
import random
import base64 as encoder

nums = random.sample(range(255), 10)

data = bytes(nums)
data

#s = str(data, encoding="utf8")  # This fails!

s = encoder.b64encode(data).decode("utf8")

outs.write(s)
outs.close()

In [17]:
import base64
with open("/tmp/test.dat") as infile:
    content = infile.read()

data = base64.b64decode(content)
print(data)
print(list(data))

b'\xa4\xb1\xf6A\x05\x06a\x80\xb2\xae'
[164, 177, 246, 65, 5, 6, 97, 128, 178, 174]


#### RESTful architecture (overview)

Example:
    endpoint: /photos
    POST /photos?action=add  -> Create a new resource
    GET  /photos?action=list -> Retrieve all photos
    POST /photos?action=update&photo_id=112  -> Update a photo 
    GET  /photos?action=delete&phto_id=112 -> Delete a photo

The above semantics does not follow a "pure" RESTful architecture pattern - as they embed the actions within a query string

In RESTful architecture - we must focus on "resources".

Resource end-points are of two types:
   1. Collection of resources (resource collection endpoints)
      - Supported HTTP verbs: GET, POST
   2. A singular resource

Example:
 - Resource collection end-points
   /photos   # A plural / collective nouns indicate a resource collection
             # endpoint
   /album    # Another example of a resource collection endpoint

   GET /photos # Retrieve all photos as a list
   GET /photos?start=10&limit=5  # Retrieve a subset of photos as a list
   - The expected return value would always be a list

   POST /photos   # Create / Insert / Add a new resource into the
                  # collection

 - Singular resource
   /photos/123   # Always ends with a unique id to represent the resource

   GET /photos/123 # Retrieve a specific resource based on the id
   DELETE /photos/123 # Delete a specific resource based on the id
   PUT /photos/123    # Replace a resource for the given id
   PATCH /photos/123  # Update a resource for the given id





In [32]:
# OpenWeatherMap GeoCoding API example

API_KEY = "932c152d6ff8d185bfdd9d2a5f8e33e4"

owm_gc_ep = "http://api.openweathermap.org/geo/1.0/direct"
owm_gc_params = {
    "q": "Chennai", 
    "limit": "1", 
    "appid": API_KEY
}

#owm_wm_ep = "https://api.openweathermap.org/data/3.0/onecall"
owm_wm_ep = "https://api.openweathermap.org/data/2.5/weather"
owm_wm_params = {
    "appid": API_KEY,
    "units": "metric",
}

import requests

res = requests.get(owm_gc_ep, params=owm_gc_params)
if res.ok and "application/json" in res.headers["Content-Type"]:
    data = res.json()
    #print(data)
    lat, lon = data[0]["lat"], data[0]["lon"]
    print(f"Getting weather for {lat=}, {lon=}")

    owm_wm_params["lat"] = lat
    owm_wm_params["lon"] = lon

res = requests.get(owm_wm_ep, params=owm_wm_params)
if res.ok and "application/json" in res.headers["Content-Type"]:
    data = res.json()
    #print(data)
    print(data["main"]["temp"])


Getting weather for lat=13.0836939, lon=80.270186
34.06


In [30]:
res.content

b'{"cod":401, "message": "Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info."}'

In [22]:
res.status_code

200

In [23]:
res.headers

{'Server': 'openresty', 'Date': 'Tue, 15 Apr 2025 05:54:24 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '543', 'Connection': 'keep-alive', 'X-Cache-Key': '/geo/1.0/direct?q=chennai', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST'}

In [24]:
res.json()

[{'name': 'Chennai',
  'local_names': {'fr': 'Chennai',
   'pl': 'Ćennaj',
   'de': 'Chennai',
   'ur': 'چنئی',
   'hi': 'चेन्नई',
   'ru': 'Ченнаи',
   'ml': 'ചെന്നൈ',
   'cs': 'Čennaí',
   'en': 'Chennai',
   'mr': 'चेन्नई',
   'ja': 'チェンナイ',
   'uk': 'Ченнаї',
   'bn': 'চেন্নাই',
   'ko': '첸나이',
   'he': "צ'נאי",
   'te': 'చెన్నై',
   'lt': 'Čenajus',
   'ar': 'تشيناي',
   'kn': 'ಚೆನ್ನೈ',
   'ta': 'சென்னை',
   'tr': 'Madras',
   'zh': '金奈'},
  'lat': 13.0836939,
  'lon': 80.270186,
  'country': 'IN',
  'state': 'Tamil Nadu'}]